# ATLAS Simulated Data Script

##### Version 1.2 (Size: 10623319 Rows by 8 Columns)


This version contains the full dataset but only 8 columns picked out by Ari for Spencer. 
 
### Notes:

## $ V_{mag} = 5\log_{10}(rt) + H_{mag} + f(\alpha)$
    
Where:
                      
$V_{mag} =$ Apparent Magnitude<br> 
$r =$ The distance from the asteroid and the Sun<br>
$t =$ The distance between the asteroid and Earth<br>
$H_{mag} =$ The Absolute Magnitude (Here we use 25)<br> 
$\alpha =$ The angle between the lines of sight to Earth and the Sun as seen from the asteroid<br>
$f(\alpha) =$ The phase function that depends on the movements of the NEO (Phase Effect)<br>

### Refrences:

* Vmag (Apparent Magnitude) of Sun = -26.7 <br>
* Hmag (Absolute Magnitude) of Sun = 4.83 

=============================================================================================================================================

#### Imports

In [1]:
import numpy as np
import pandas as pd

#### Data

In [2]:
# File directory to location of data

directory = '/Users/Spencer/Desktop/ATLAS/ATLAS_Research/Data/' 

In [11]:
# Read in data through Numpy
# Header: asteroid_id, geodist, heliodist, angvel, phaseeffect, geodistclose, phaseangclose, encountervel

data_np= np.genfromtxt(directory + 'dyn8merged02spencer01.txt', dtype='float', delimiter='', skip_header = 1,
                names = ['asteroid_id'
                        ,'geodist'
                        ,'heliodist'
                        ,'angvel'
                        ,'phaseeffect'
                        ,'geodistclose'
                        ,'phaseangclose'
                        ,'encountervel'],)

In [12]:
# Read in data through Pandas

df_pandas = pd.read_csv(directory + 'dyn8merged02spencer01.txt', engine='python', sep=' ',
                  names=['asteroid_id'
                        ,'geodist'
                        ,'heliodist'
                        ,'angvel'
                        ,'phaseeffect'
                        ,'geodistclose'
                        ,'phaseangclose'
                        ,'encountervel'])



#### A Function to find the Vmag of an asteroid

In [13]:
def ap_mag(H,helio_d,geo_d,phi):
    
    '''
    Function to calculate the apparent magnitude of an asteroid.
    
    *H is equal to Absolute Magnitude of asteroid (unitless) (float or int) 
    
    *Be sure helio_d and geo_d are (AU) (float)
    
    *Phi must be greater or equal to zero (unitless) (float)
    
    '''
    
    return H + 5 * np.log10(helio_d * geo_d) + phi

In [14]:
vmags = ap_mag(25,data_np['heliodist'],data_np['geodist'],data_np['phaseeffect'])

In [15]:
# asteroid ids vmag<=19

df_less_19 = data_np['asteroid_id'][vmags<=19]
print('The number of asteroids whos apparent magnitudes are <= 19 is:', len(df_less_19))

The number of asteroids whos apparent magnitudes are <= 19 is: 115074


# Streaked vs. Non-Streaked

### Required apparent magnitudes to be deteced by ATLAS

Non-Streaked vmag < 19 <br>
Streaked vmag = m_lim (See equation below) 

## $M_{lim}=19-2.5\log(e)$ <br>

##### Where: 

$M_{lim}$= Apparent magnitude needed to detect a streaked asteroid <br>
$e=\frac{l}{2r}$ <br>
$l$= Length of the streak <br>
$r$= Radius of streak

##### How to find $l$:

#### $l=E(Seconds) \cdot V_a(\frac{Degree}{Day}) \cdot \frac{3600}{1}(\frac{ArcSeconds}{Degree})\cdot \frac{1}{86400}(\frac{Day}{Seconds})$

##### Where:

$E$= Exsposure time (Seconds) <br>
$V_a$= Angular Velocity (Degrees per day) <br>

##### Constants:

$E$= 30 Seconds. This is the typical exposure time for ATLAS images. <br>
$2r$ = 4 Arc Seconds.

=============================================================================================================================================

##### Calculating  $l$ for the entire df (dataframe):

In [33]:
df_pandas['length'] = 30*df_pandas['angvel']*3600*(1/86400)

##### Calculating the $M_{lim}$ of the entire df (dataframe):


In [17]:
df_pandas['m_lim']= 19-2.5*np.log10(df_pandas['length']/4)

In [22]:
df_pandas['m_lim'].describe()

count    1.062332e+07
mean     1.861948e+01
std      8.061432e-01
min      1.263524e+01
25%      1.809103e+01
50%      1.858147e+01
75%      1.909352e+01
max      2.706371e+01
Name: m_lim, dtype: float64

In [31]:
# Number of streaked asteroids that would have to be less than or equal to 19 magnitude to be detected
m_lim_19 = df_pandas['m_lim'][df_pandas['m_lim']<=19]
m_lim_19.describe()

count    7.547481e+06
mean     1.823217e+01
std      5.245486e-01
min      1.263524e+01
25%      1.791493e+01
50%      1.830845e+01
75%      1.864298e+01
max      1.900000e+01
Name: m_lim, dtype: float64